<a href="http://landlab.github.io"><img style="float: left" src="../../landlab_header.png"></a>

# Mapping values between grid elements

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>)
<hr>

Imagine that you're using Landlab to write a model of shallow water flow over terrain. A natural approach is to place your scalar fields, such as water depth, at the nodes. You then place your vector fields, such as water surface gradient, flow velocity, and discharge, at the links. But your velocity depends on both slope and depth, which means you need to know the depth at the links too. How do you do this?

This tutorial introduces *mappers*: grid functions that map quantities defined on one set of elements (such as nodes) onto another set of elements (such as links). As you'll see, there are a variety of mappers available.



## Mapping from nodes to links

For the sake of example, we'll start with a simple 3-row by 4-column raster grid. The grid will contain a scalar field called `water__depth`, abbreviated `h`. We'll populate it with some example values, as follows:

In [1]:
from landlab import RasterModelGrid
import numpy as np
mg = RasterModelGrid((3, 4), xy_spacing=100.0)
h = mg.add_zeros('surface_water__depth', at='node')
h[:] = 7 - np.abs(6 - np.arange(12))

For the sake of visualizing values at nodes on our grid, we'll define a handy little function:

In [2]:
def show_node_values(mg, u):
    for r in range(mg.number_of_node_rows - 1, -1, -1):
        for c in range(mg.number_of_node_columns):
            print(int(u[c + (mg.number_of_node_columns * r)]), end=' ')
        print()

In [3]:
show_node_values(mg, h)

5 4 3 2 
5 6 7 6 
1 2 3 4 


Let's review the numbering of nodes and links. The lines below will print a list that shows, for each link ID, the IDs of the nodes at the link's tail and head:

In [4]:
for i in range(mg.number_of_links):
    print(i, mg.node_at_link_tail[i], mg.node_at_link_head[i])

0 0 1
1 1 2
2 2 3
3 0 4
4 1 5
5 2 6
6 3 7
7 4 5
8 5 6
9 6 7
10 4 8
11 5 9
12 6 10
13 7 11
14 8 9
15 9 10
16 10 11


### Finding the mean value between two nodes on a link

Suppose we want to have a *link-based* array, called *h_edge*, that contains water depth at locations between adjacent pairs of nodes. For each link, we'll simply take the average of the depth at the link's two nodes. To accomplish this, we can use the `map_mean_of_link_nodes_to_link` grid method. At link 8, for example, we'll average the *h* values at nodes 5 and 6, which should give us a depth of (6 + 7) / 2 = 6.5:

In [5]:
h_edge = mg.map_mean_of_link_nodes_to_link('surface_water__depth')
for i in range(mg.number_of_links):
    print(i, h_edge[i])

0 1.5
1 2.5
2 3.5
3 3.0
4 4.0
5 5.0
6 5.0
7 5.5
8 6.5
9 6.5
10 5.0
11 5.0
12 5.0
13 4.0
14 4.5
15 3.5
16 2.5


### What's in a name?

The mapping functions have long names, which are designed to make it as clear as possible to understand what each function does. All the mappers start with the verb *map*. Then the relationship is given; in this case, we are looking at the *mean*. Then the elements from which a quantity is being mapped: we are taking values from *link nodes*. Finally, the element to which the new values apply: *link*.

### Mapping minimum or maximum values

We can also map the minimum value of *h*:

In [6]:
h_edge = mg.map_min_of_link_nodes_to_link('surface_water__depth')
for i in range(mg.number_of_links):
    print(i, h_edge[i])

0 1.0
1 2.0
2 3.0
3 1.0
4 2.0
5 3.0
6 4.0
7 5.0
8 6.0
9 6.0
10 5.0
11 4.0
12 3.0
13 2.0
14 4.0
15 3.0
16 2.0


... or the maximum:

In [7]:
h_edge = mg.map_max_of_link_nodes_to_link('surface_water__depth')
for i in range(mg.number_of_links):
    print(i, h_edge[i])

0 2.0
1 3.0
2 4.0
3 5.0
4 6.0
5 7.0
6 6.0
7 6.0
8 7.0
9 7.0
10 5.0
11 6.0
12 7.0
13 6.0
14 5.0
15 4.0
16 3.0


### Upwind and downwind

Numerical schemes often use *upwind differencing* or *downwind differencing*. For example, finite difference schemes for equations that include advection may use "upwind" rather than centered differences, in which a scalar quantity (our *h* for example) is taken from whichever side is upstream in the flow field.

How do we know the flow direction? If the flow is driven by the gradient in some scalar field, such as pressure or elevation, one approach is to look at the values of this scalar on either end of the link: the end with the higher value is upwind, and the end with the lower value is downwind.

Suppose for example that our water flow is driven by the water-surface slope (which is often a good approximation for the *energy slope*, though it omits the kinetic energy). Let's define a bed-surface elevation field *z*:

In [8]:
z = mg.add_zeros('topographic__elevation', at='node')
z[:] = 16 - np.abs(7 - np.arange(12))
show_node_values(mg, z)

15 14 13 12 
13 14 15 16 
9 10 11 12 


The water-surface elevation is then the sum of *h* and *z*:

In [9]:
w = z + h
show_node_values(mg, w)

20 18 16 14 
18 20 22 22 
10 12 14 16 


For every link, we can assign the value of *h* from whichever end of the link has the greater *w*:

In [10]:
h_edge = mg.map_value_at_max_node_to_link(w, h)
for i in range(mg.number_of_links):
    print(i, h_edge[i])

0 2.0
1 3.0
2 4.0
3 5.0
4 6.0
5 7.0
6 6.0
7 6.0
8 7.0
9 6.0
10 5.0
11 6.0
12 7.0
13 6.0
14 5.0
15 4.0
16 3.0


Consider the middle two nodes (5 and 6). Node 6 is higher (22 versus 20). Therefore, the link between them (link 8) should be assigned the value of *h* at node 6. This value happens to be 7.0.

Of course, we could also take the value from the *lower* of the two nodes, which gives link 8 a value of 6.0:

In [11]:
h_edge = mg.map_value_at_min_node_to_link(w, h)
for i in range(mg.number_of_links):
    print(i, h_edge[i])

0 1.0
1 2.0
2 3.0
3 1.0
4 2.0
5 3.0
6 4.0
7 5.0
8 6.0
9 6.0
10 5.0
11 4.0
12 3.0
13 2.0
14 4.0
15 3.0
16 2.0


### Heads or tails?

It is also possible to map the scalar quantity at either the head node or the tail node to the link:

In [12]:
h_edge = mg.map_link_head_node_to_link('surface_water__depth')
for i in range(mg.number_of_links):
    print(i, h_edge[i])

0 2.0
1 3.0
2 4.0
3 5.0
4 6.0
5 7.0
6 6.0
7 6.0
8 7.0
9 6.0
10 5.0
11 4.0
12 3.0
13 2.0
14 4.0
15 3.0
16 2.0


In [13]:
h_edge = mg.map_link_tail_node_to_link('surface_water__depth')
for i in range(mg.number_of_links):
    print(i, h_edge[i])

0 1.0
1 2.0
2 3.0
3 1.0
4 2.0
5 3.0
6 4.0
7 5.0
8 6.0
9 7.0
10 5.0
11 6.0
12 7.0
13 6.0
14 5.0
15 4.0
16 3.0


### Simple example using centered water depth
The following implements one time-step of a linear-viscous flow model, in which flow velocity is calculated at the links, and the depth at each link is taken as the mean of depth at the two bounding nodes. To make the flow a little tamer, we'll have our fluid be hot, low viscosity basaltic lava instead of water, with a dynamic viscosity of 100 Pa s.

In [14]:
gamma = 25000.0  # unit weight of fluid, N/m2
viscosity = 100.0  # dynamic viscosity in Pa s
grad = mg.calc_grad_at_link(w)
h_edge = mg.map_mean_of_link_nodes_to_link(h)
vel = -(gamma / (3.0 * viscosity)) * h_edge * h_edge * grad
for ln in range(mg.number_of_links):
    print(ln, h_edge[ln], grad[ln], vel[ln])

0 1.5 0.02 -3.75
1 2.5 0.02 -10.4166666667
2 3.5 0.02 -20.4166666667
3 3.0 0.08 -60.0
4 4.0 0.08 -106.666666667
5 5.0 0.08 -166.666666667
6 5.0 0.06 -125.0
7 5.5 0.02 -50.4166666667
8 6.5 0.02 -70.4166666667
9 6.5 0.0 -0.0
10 5.0 0.02 -41.6666666667
11 5.0 -0.02 41.6666666667
12 5.0 -0.06 125.0
13 4.0 -0.08 106.666666667
14 4.5 -0.02 33.75
15 3.5 -0.02 20.4166666667
16 2.5 -0.02 10.4166666667


I'm not sure I love the idea of a 5-m thick lava flow moving at over 100 m/s! (I guess we can take some comfort from the thought that turbulence would probably slow it down)

How different would the numerical solution be using an upwind scheme for flow depth? Let's find out:

In [15]:
h_edge = mg.map_value_at_max_node_to_link(w, h)
vel = -(gamma / (3.0 * viscosity)) * h_edge * h_edge * grad
for ln in range(mg.number_of_links):
    print(ln, h_edge[ln], grad[ln], vel[ln])

0 2.0 0.02 -6.66666666667
1 3.0 0.02 -15.0
2 4.0 0.02 -26.6666666667
3 5.0 0.08 -166.666666667
4 6.0 0.08 -240.0
5 7.0 0.08 -326.666666667
6 6.0 0.06 -180.0
7 6.0 0.02 -60.0
8 7.0 0.02 -81.6666666667
9 6.0 0.0 -0.0
10 5.0 0.02 -41.6666666667
11 6.0 -0.02 60.0
12 7.0 -0.06 245.0
13 6.0 -0.08 240.0
14 5.0 -0.02 41.6666666667
15 4.0 -0.02 26.6666666667
16 3.0 -0.02 15.0


Even scarier.

In any event, this example illustrates how you can use Landlab's mapping functions to build mass-conservation models in which the flow rate depends on a gradient and a scalar, both of which can be evaluated at links.